In [1]:
# Install the OpenAI library if you haven't already
#pip install openai

In [2]:
import openai
import os
import logging
import datetime
import pytz

In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import re
import pickle

from scipy import sparse



In [4]:
# Set your OpenAI API key
api_key = 'openai key'  # Replace with your actual API key
#os.environ["OPENAI_API_KEY"] = api_key

In [5]:


# Function to read content from a readme.md file
def read_readme_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        return None

# Function to read content from a readme.md file
def get_read_content(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        return None

# Function to ask questions using GPT-3 API
def ask_gpt_question(prompt):
    #openai.api_key = environ['OPENAI_API_KEY'],
    openai.api_key = api_key
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[
        {
            "role": "system",
            "content": prompt,
        }
    ],
        max_tokens=150,  # You can adjust this value to control the response length
        #prompt=prompt,
        stop=None,
        temperature=0.7,
    )
    #return response.choices[0].text.strip()
    return(response.choices[0].message.content)



    # Function to ask questions using GPT-3 API
def ask_gpt_question_adv_prompt(sysprompt, userprompt):
    #openai.api_key = environ['OPENAI_API_KEY'],
    openai.api_key = api_key
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[
        {
            "role": "system",
            "content": sysprompt,
        },
        {
            "role": "user",
            "content": userprompt,

        }
    ],
        max_tokens=150,  # You can adjust this value to control the response length
        #prompt=prompt,
        stop=None,
        temperature=0.7,
    )
    #return response.choices[0].text.strip()
    return(response.choices[0].message.content)



In [7]:
#v2 prompt

def ask_llm_v4(projectname, readme_content):
    
    #if not readme_content:
    if True:
        # Ask the first question: "Is this scientific software? Yes or no?" 
        #our prompt before readme, is 1300 characters ~269 tokens, approx 4:1
        #overall, we can have 16384 chars approx, 4096 tokens
        #Lets reserve 1300 characters for output
        #so total prompt can only be 13784 characters, lets trim prompts after 13000 chars
        question1_prompt = f"""We are analyzing READMEs in software repositories to compile a dataset of scientific application software projects. You'll receive a README to review and answer questions about, using the file and your knowledge. Scientific application software is defined as domain-specific science and engineering software, excluding general software like tools and libraries.  Science support software is defined as non domain specific tools and libraries that are used to support scientific applications or research. Some examples include Numerical libraries (includes optimization, statistics, simulation, e.g., numpy)
Data collection (e.g., data acuisition controls for experiments or portals), Visualization (interfaces to interact with, understand, and see data, plotting tools), Workflow managers
Provenance and metadata collection tools. Research software is software that is used to generate, process or analyse results that you intend to appear in a publication (either in a journal, conference paper, monograph, book or thesis).Please answer the following questions for us…
    
   1. Is this domain-specific scientific application software (answer Yes or No only)?
   2. Does readme mention a publication or paper or research funding related to this software package (answer Yes or No only)?
   3. Is this research software (answer Yes or No only)?
   4. Is this Science support software (answer Yes or No only)?
   5. In no more than one or two sentences, please describe the purpose of this software project (delimit with “” quotes).
   6. Make a list of no more than 5-6 short keywords or phrases that describe the domain of the software ([keyword/phrase 1, keyword/phrase 2, etc.]).

    I'll provide an output template; format your answers on a single line, comma-delimited: #1 | #2 | #3 | #4 | #5 | #6 . No more text besides this.
    Example responses:
    
    Yes | No | Yes | Yes| “Mantid is a software application for manipulating and analyzing neutron and muon scattering data. It is designed for high-performance computing and visualization.”|["neutron scattering", "scientific analysis", "simulation", "high-performance computing", "visualization"]

    No | No | No  |No | “BudgetTracker is a software tool for anaging personal finances. It allows users to add categories and track expenses.”| ["personal budget", "finance planner","Expense tracking"]

    . given readme:{readme_content}"""

        # Display metadata
        #print("Project ", projectname)
        #print ("input_prompt", question1_prompt )
        question1_prompt_truncated = question1_prompt[:10000]
        try:           
           answer1 = ask_gpt_question(question1_prompt_truncated)

        except:
            print("Project ", projectname)
            logging.info("Project ", projectname)
            logging.exception('')
            answer1 = """Exception occured |Exception occured |Exception occured |Exception occured |Exception occured | """
    
        # Ask the second question: "List the top 10 contextual topics for this readme.md"
       
    else:
        print("Readme file not found or empty.")
        logging.info("Readme file not found or empty.")
    return(answer1)

In [9]:
#loop with files

In [10]:
est_tz = pytz.timezone('America/New_York')

run_start_time =  datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))  # Eastern St
print(f'Run started. Time : {run_start_time}')
logging.info(f'Run started. Time : {run_start_time}')
# Loop through files from 1 to 50
for file_num in range(50, 51):
        # Load the entire DataFrame contained in the pickle file
    file_path = f"./take2/unzipping/processed_data/processeddata{file_num}.pkl"
    with open(file_path, "rb") as f:
        df = pickle.load(f)
        if(file_num<50):
            # Ensure the DataFrame has 6994 rows
            total_rows = 6994      
        else:
            total_rows = 6946

        # Calculate the number of DataFrames needed with 1000 rows each and the remaining rows
        #num_dataframes = total_rows // 1000
        #remaining_rows = total_rows % 1000

        num_dataframes = total_rows // 250
        remaining_rows = total_rows % 250
        
         # Split the DataFrame into DataFrames of 1000 rows each
        #dataframes = [df.iloc[i:i+1000] for i in range(0, num_dataframes * 1000, 1000)]
        dataframes = [df.iloc[i:i+250] for i in range(0, num_dataframes * 250, 250)]
        
        # Add the last DataFrame with the remaining rows
        if remaining_rows > 0:
            dataframes.append(df.iloc[num_dataframes * 200:])

        file_start_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))  # Eastern Standard Time (EST)
        print(f"Processing p{file_num}  Time: {file_start_time}")
        logging.info(f"Processing p{file_num}  Time: {file_start_time}")
          # Loop through the DataFrames
        for i, df_chunk in enumerate(dataframes, start=1):
            # Initialize a dictionary to store all results
            scidatafull = {}
            chunk_start_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))  # Eastern Standard Time (EST)
            print(f"Processing p{file_num} - df{i}... Time: {chunk_start_time}")
            print(df_chunk.shape)
            logging.info(f"Processing p{file_num} - df{i}... Time: {chunk_start_time}")
            logging.info(df_chunk.shape)
            logging.info
            df_chunk.sample(2)
            df_chunk.head(3)
            for index, row in df_chunk.iterrows():
                try:
                    # print(df_chunk.shape)
                    # df_chunk.sample(2)
                    # df_chunk.head(3)
                    scidata = {}
                    scidata['ProjectID'] = row['ProjectID']
                    print(f"Processing project : { scidata['ProjectID']} ...")
                    logging.info(f"Processing project : { scidata['ProjectID']} ...")
                    llm_answer = ask_llm_v4(row['ProjectID'], str(row['readme']))
                    elements = llm_answer.split('|')
    
                    #scidata['isScientificSoftware'] = elements[0].strip() if len(elements) > 0 else None
                    #scidata['short_description'] = elements[1] if len(elements) > 1 else None
                    #scidata['keywords'] = elements[2] if len(elements) > 2 else None

                    scidata['isScientificAppSoftware'] = elements[0].strip() if len(elements) > 0 else None
                    scidata['mentionsPaperOrFunding'] = elements[1].strip() if len(elements) > 1 else None
                    scidata['isResearchSoftware'] = elements[2].strip() if len(elements) > 2 else None
                    scidata['isScienceSupportSoftware'] = elements[3].strip() if len(elements) > 3 else None
                    scidata['short_description'] = elements[4] if len(elements) > 4 else None
                    scidata['keywords'] = elements[5] if len(elements) > 5 else None
    
                    scidata['full_answer'] = llm_answer
    
                    scidatafull[row['ProjectID']] = scidata
                except Exception as e:
                    print(f"Error processing row {index} in p{file_num} - df{i}: {str(e)}")
                    print("Project ", projectname)
                    logging.info(f"Error processing row {index} in p{file_num} - df{i}: {str(e)}")
                    logging.info("Project ", projectname)
                    logging.exception('')
                    answer1 = """Exception occured |Exception occured |Exception occured |"""
                    continue

                # Create a new DataFrame with the processed values
            print(f'>>Finished processing p{file_num} - df{i}')
            logging.info(f'>>Finished processing p{file_num} - df{i}')
            sci_df = pd.DataFrame(scidatafull.values())
        
                # Merge with the original DataFrame
            processed_df = pd.merge(sci_df, df_chunk, on='ProjectID', how='inner')
            print(f'Merged p{file_num} - df{i} with metadata, shape: {processed_df.shape}')
            logging.info(f'Merged p{file_num} - df{i} with metadata, shape: {processed_df.shape}')
            metadata_columns_to_exclude = ['Core', 'RootFork', 'readme' ]
            # want to take out readme, commitid and core, since email data
            processed_df = processed_df.drop(columns=metadata_columns_to_exclude)
            processed_df = processed_df[['ProjectID','url', 'isScientificAppSoftware', 'mentionsPaperOrFunding', 'isResearchSoftware', 'isScienceSupportSoftware','keywords','short_description',  
       'full_answer','NumForks', 'NumStars','NumFiles', 'NumAuthors',
       'NumCore', 'Gender', 'NumCommits', 'NumBlobs','FileInfo',
      'NumActiveMon',  'LatestCommitDate', 'EarliestCommitDate',  'CommunitySize', 'blobId', 'commitID']]
            processed_df = processed_df.rename(columns={'FileInfo': 'CodeLanguages', 'NumCore': 'NumCoreAuthors', 'Gender': 'AuthorsGender'})
            print('we merged with metadata')
            logging.info('we merged with metadata')
        
                # Save the processed DataFrame to pickle and CSV files
            processed_df.to_pickle(f'./take2/unzipping/processed_data/scidata/finalscidata-{file_num}-{i}.pkl')
            processed_df.to_csv(f'./take2/unzipping/processed_data/scidata/finalscidata-{file_num}-{i}.csv')
            print(processed_df.shape)            
            print(f'Saved p{file_num} - df{i} as pickle and CSV')
            logging.info(processed_df.shape)            
            logging.info(f'Saved p{file_num} - df{i} as pickle and CSV')

    file_end_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))  # Eastern St
    print(f'File p{file_num} finished. Time : {file_end_time}')
    logging.info(f'File p{file_num} finished. Time : {file_end_time}')

run_end_time =  datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))  # Eastern St
print(f'Run finished. Time : {run_end_time}')
logging.info(f'Run finished. Time : {run_end_time}')

Run started. Time : 2023-12-01 09:41:59.258110-05:00
Processing p50  Time: 2023-12-01 09:41:59.690353-05:00
Processing p50 - df1... Time: 2023-12-01 09:41:59.690677-05:00
(250, 20)
Processing project : Naicah_Retargeting_app ...
Processing project : hq450_fancyss ...
Processing project : jardenilmark_wiwu-user ...
Processing project : bitbucket.org_minoba_javaintroprogrammingassignments ...
Processing project : PixarV_jrecord ...
Processing project : jaenster_charon ...
Processing project : PHSKC-APDE_rads ...
Processing project : hselasky_isdn4bsd ...
Processing project : Songmu_p5-Riji ...
Processing project : nornir-automation_nornir ...
Processing project : kohterai_portfolio-website ...
Processing project : jnordberg_dsteem ...
Processing project : jranke_mkin ...
Processing project : wwagner4_gaia ...
Processing project : WIAS-BERLIN_qMRI ...
Processing project : GoogleChromeLabs_ui-element-samples ...
Processing project : UCDavisLibrary_main-wp-website ...
Processing project : i

In [11]:
#done